# First Machine Learning Model

Sarah Braverman

In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split


#### When done, put the imputing and OHE code here to be able to create and use a model

**Model Purpose:** 
**A model to predict whether the avalanche will hit the rail depending on the conditions** 

List: 

Do OHE, imputing etc. 

Test parameters, what are the best settings? Max_depth, n_estimators etc.

Figure out how to find out which features impact the rail the most
 
Train model on train data, fit on all data, train on all data, see how accurate it can be